<a href="https://colab.research.google.com/github/tominaga-RS/AI_PYTHON_WS/blob/main/NN_Traffic_Light.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RSリモート環境では「ノートブックを新規作成」が不可のため、
当ファイルをコピーしてお使いください。

In [ ]:
from google.colab import files
import csv
import datetime

# 学習データフォルダ
LEARNING_DATA_FOLDER_PATH = ".\\Traffic_Light_Work\\learning_data"
# 出力フォルダ
OUTPUT_FILE_PATH = ".\\Traffic_Light_Work\\result.csv"

# マッチ箱役の変数を定義(MB = MatchBox)
# 最終判定用のマッチ箱役変数
MB_Answer = 8

# マッチ箱1つ分を表すクラス
class MatchBox:
    # コンストラクタ
    def __init__(self, value=None, weight=None):
        self.value = value if value is None else 0 # 値
        self.weight = weight if weight is None else 0 # 重み

    # インスタンス変数外部アクセス用メソッド
    def set_value(self, value):
        self.value = value
    def get_value(self):
        return self.value

    def set_weight(self, weight):
        self.weight = weight
    def get_weight(self):
        return self.weight
    
# 画像の特定範囲の代表点を評価するためのマッチ箱役クラス
class ImageCell:
    # コンストラクタ
    def __init__(self):
        self.MB_Color_R = MatchBox() # 画像の赤色のマッチ箱
        self.MB_Color_G = MatchBox() # 画像の緑色のマッチ箱
        self.MB_Color_B = MatchBox() # 画像の青色のマッチ箱
        self.MB_Brightness = MatchBox() # 画像の明るさのマッチ箱
        self.MB = MatchBox() # 代表点全体のマッチ箱

    # インスタンス変数外部アクセス用メソッド    
    def set_MB_Color_R(self, MatchBox):
        self.MB_Color_R = MatchBox
    def get_MB_Color_R(self):
        return self.MB_Color_R
    
    def set_MB_Color_G(self, MatchBox):
        self.MB_Color_G = MatchBox
    def get_MB_Color_G(self):
        return self.MB_Color_G

    def set_MB_Color_B(self, MatchBox):
        self.MB_Color_B = MatchBox
    def get_MB_Color_B(self):
        return self.MB_Color_B

    def set_MB_Brightness(self, MatchBox):
        self.MB_Brightness = MatchBox
    def get_MB_Brightness(self):
        return self.MB_Brightness

    def set_MB(self, MatchBox):
        self.MB = MatchBox
    def get_MB(self):
        return self.MB

# 学習経過を保持しておくマッチ箱クラスインスタンス
MB_ImageCells = []
for num in range(3):
    MB_ImageCells.append(ImageCell())

#==================================================================#
# 信号機の色の並びを機械学習させ、スクリーンショットで取得した二次元画像が
# 信号機の色の並びかを判定するプログラム
#==================================================================#

# 一回試行分の関数を定義
# 引数：JudgedCells 判定対象画像代表点の情報を表す2次元リスト
#       {{R,G,B,明るさ},
#        {R,G,B,明るさ},
#        {R,G,B,明るさ}}
#       の形式
#       answer 
def trial(JudgedCells, answer):
    # 最終判定
    global MB_Answer
    
    # 代表点毎のマッチ箱
    global MB_ImageCells
    
    # 変数を初期化
    totalMatch = 0
    matchAnswer = False

    print ["Correct Answer:",answer]
    
    # 代表点毎のマッチ箱が興奮するか否かを判定
    Judge_excitement = [] # 興奮するかしないかを2次元リストで管理する
    MB_total_weight = 0 # 興奮した全代表点の重みの合計値
    for row_num in range(len(JudgedCells)):
        # 代表点毎のマッチ箱の値を閾値として、判定対象マッチ箱の値がそれを超えたら興奮したことにする
        excitement = [] # 各代表点の興奮状態を格納するリスト
        total_weight = 0 # 興奮した代表点の各マッチ箱の重みを合計した重み
        
        # 赤色のマッチ箱が興奮するかを判定
        if(JudgedCells[row_num][0] > MB_ImageCells[row_num].get_MB_Color_R().get_value()):
            excitement.append(1)
            total_weigtht = total_weight + MB_ImageCells[row_num].get_MB_Color_R().get_weight()
        else:
            excitement.append(0)
            
        # 緑色のマッチ箱が興奮するかを判定
        if(JudgedCells[row_num][1] > MB_ImageCells[row_num].get_MB_Color_G().get_value()):
            excitement.append(1)
            total_weigtht = total_weight + MB_ImageCells[row_num].get_MB_Color_G().get_weight()
        else:
            excitement.append(0)
            
        # 青色のマッチ箱が興奮するかを判定
        if(JudgedCells[row_num][2] > MB_ImageCells[row_num].get_MB_Color_B().get_value()):
            excitement.append(1)
            total_weigtht = total_weight + MB_ImageCells[row_num].get_MB_Color_B().get_weight()
        else:
            excitement.append(0)
            
        # 明るさのマッチ箱が興奮するかを判定
        if(JudgedCells[row_num][3] > MB_ImageCells[row_num].get_MB_Brightness().get_value()):
            excitement.append(1)
            total_weigtht = total_weight + MB_ImageCells[row_num].get_MB_Brightness().get_weight()
        else:
            excitement.append(0)

        # マッチ箱全体として興奮するか判定
        if(total_weigtht > MB_ImageCells[row_num].get_MB().get_value()):
            excitement.append(1)
            MB_total_weight = MB_total_weight + MB_ImageCells[row_num].get_MB().get_weight()
        else:
            excitement.append(0)
   
        Judge_excitement.append(excitement)

         
    # マッチ箱の答え
    # 全代表点の重みの合計値が閾値を超えたら信号機だと判断する
    if(MB_total_weight > MB_Answer):
        matchAnswer = True
    else:
        matchAnswer = False
        
    print ["Match Answer:", matchAnswer]

    #==============#
    # ペナルティ設定 #
    #==============#
    # 信号機の並びなのに信号機でないと判断した場合
    if(answer):
        if(not matchAnswer):
            # ペナルティ設定:合計の閾値を減らす
            MB_Answer = MB_Answer - 1
            
            # ペナルティ設定:代表点のマッチ箱の重みを増やす
            for row_num in range(len(Judge_excitement):
                # マッチ箱全体として興奮していたかを確認
                if(Judge_excitement[row_num][4]):
                    # 興奮していた場合、合計の判定に渡すマッチ棒の数が少なかったと考える
                    # 重み（合計のマッチ箱に送るマッチ棒の数）を増やす
                    tmpMB = MatchBox(MB_ImageCells[row_num].get_MB().get_value(), MB_ImageCells[row_num].get_MB().get_Weight()+1)
                    MB_ImageCells[row_num].set_MB(tmpMB)
                else:
                    # 興奮していなかった場合、本当は興奮しなければいけなかったと考える
                    # 値（閾値）を減らす
                    tmpMB = MatchBox(MB_ImageCells[row_num].get_MB().get_value()-1, MB_ImageCells[row_num].get_MB().get_Weight())
                    MB_ImageCells[row_num].set_MB(tmpMB)
                    # 赤の値を設定
                    if(Judge_excitement[row_num][0]):
                        # 興奮していた場合、代表点のマッチ箱に渡すマッチ棒の数が少なかったと考える
                        tmpMBR = MatchBox(MB_ImageCells[row_num].get_MB_Color_R().get_value(), MB_ImageCells[row_num].get_MB_Color_R().get_Weight()+1)
                        MB_ImageCells[row_num].set_MB_Color_R(tmpMBR)
                    else:
                        # 興奮していなかった場合、本当は興奮しなければいけなかったと考える
                        # 値（閾値）を減らす
                        tmpMBR = MatchBox(MB_ImageCells[row_num].get_MB_Color_R().get_value()-1, MB_ImageCells[row_num].get_MB_Color_R().get_Weight())
                        MB_ImageCells[row_num].set_MB_Color_R(tmpMBR)
                        
                    # 緑の値を設定
                    if(Judge_excitement[row_num][1]):
                        # 興奮していた場合、代表点のマッチ箱に渡すマッチ棒の数が少なかったと考える
                        tmpMBG = MatchBox(MB_ImageCells[row_num].get_MB_Color_G().get_value(), MB_ImageCells[row_num].get_MB_Color_G().get_Weight()+1)
                        MB_ImageCells[row_num].set_MB_Color_G(tmpMBG)
                    else:
                        # 興奮していなかった場合、本当は興奮しなければいけなかったと考える
                        # 値（閾値）を減らす
                        tmpMBG = MatchBox(MB_ImageCells[row_num].get_MB_Color_G().get_value()-1, MB_ImageCells[row_num].get_MB_Color_G().get_Weight())
                        MB_ImageCells[row_num].set_MB_Color_G(tmpMBG)

                    # 青の値を設定
                    if(Judge_excitement[row_num][2]):
                        # 興奮していた場合、代表点のマッチ箱に渡すマッチ棒の数が少なかったと考える
                        tmpMBB = MatchBox(MB_ImageCells[row_num].get_MB_Color_B().get_value(), MB_ImageCells[row_num].get_MB_Color_B().get_Weight()+1)
                        MB_ImageCells[row_num].set_MB_Color_B(tmpMBB)
                    else:
                        # 興奮していなかった場合、本当は興奮しなければいけなかったと考える
                        # 値（閾値）を減らす
                        tmpMBB = MatchBox(MB_ImageCells[row_num].get_MB_Color_B().get_value()-1, MB_ImageCells[row_num].get_MB_Color_B().get_Weight())
                        MB_ImageCells[row_num].set_MB_Color_B(tmpMBB)

                    # 明るさの値を設定
                    if(Judge_excitement[row_num][3]):
                        # 興奮していた場合、代表点のマッチ箱に渡すマッチ棒の数が少なかったと考える
                        tmpMBL = MatchBox(MB_ImageCells[row_num].get_MB_Brightness().get_value(), MB_ImageCells[row_num].get_MB_Brightness().get_Weight()+1)
                        MB_ImageCells[row_num].set_MB_Brightness(tmpMBL)
                    else:
                        # 興奮していなかった場合、本当は興奮しなければいけなかったと考える
                        # 値（閾値）を減らす
                        tmpMBL = MatchBox(MB_ImageCells[row_num].get_MB_Brightness().get_value()-1, MB_ImageCells[row_num].get_MB_Brightness().get_Weight())
                        MB_ImageCells[row_num].set_MB_Brightness(tmpMBL) 
    # 信号機の並びでないのに信号機だと判断した場合                      
    else:
        if(matchAnswer):
            # ペナルティ設定:合計の閾値を増やす
            MB_Answer = MB_Answer + 1
            
            # ペナルティ設定:代表点のマッチ箱の重みを減らす
            for row_num in range(len(Judge_excitement):
                # マッチ箱全体として興奮していたかを確認
                if(not Judge_excitement[row_num][4]):
                    # 興奮していた場合、合計の判定に渡すマッチ棒の数が多かったと考える
                    # 重み（合計のマッチ箱に送るマッチ棒の数）を減らす
                    tmpMB = MatchBox(MB_ImageCells[row_num].get_MB().get_value(), MB_ImageCells[row_num].get_MB().get_Weight()-1)
                    MB_ImageCells[row_num].set_MB(tmpMB)
                    
                    # また、本当は興奮してはいけなかったと考える
                    # 値（閾値）を増やす
                    tmpMB = MatchBox(MB_ImageCells[row_num].get_MB().get_value()+1, MB_ImageCells[row_num].get_MB().get_Weight())
                    MB_ImageCells[row_num].set_MB(tmpMB)

                    # 赤の値を設定
                    if(Judge_excitement[row_num][0]):
                        # 興奮していた場合、代表点のマッチ箱に渡すマッチ棒の数が多かったと考える
                        tmpMBR = MatchBox(MB_ImageCells[row_num].get_MB_Color_R().get_value(), MB_ImageCells[row_num].get_MB_Color_R().get_Weight()-1)
                        MB_ImageCells[row_num].set_MB_Color_R(tmpMBR)

                        # また、本当は興奮してはいけなかったと考える
                        # 値（閾値）を増やす
                        tmpMBR = MatchBox(MB_ImageCells[row_num].get_MB_Color_R().get_value()+1, MB_ImageCells[row_num].get_MB_Color_R().get_Weight())
                        MB_ImageCells[row_num].set_MB_Color_R(tmpMBR)

                  　# 興奮していなかった場合、ペナルティなし
                        
                    # 緑の値を設定
                    if(Judge_excitement[row_num][1]):
                        # 興奮していた場合、代表点のマッチ箱に渡すマッチ棒の数が多かったと考える
                        tmpMBG = MatchBox(MB_ImageCells[row_num].get_MB_Color_G().get_value(), MB_ImageCells[row_num].get_MB_Color_G().get_Weight()-1)
                        MB_ImageCells[row_num].set_MB_Color_G(tmpMBG)

                        # また、本当は興奮してはいけなかったと考える
                        # 値（閾値）を増やす
                        tmpMBG = MatchBox(MB_ImageCells[row_num].get_MB_Color_G().get_value()+1, MB_ImageCells[row_num].get_MB_Color_G().get_Weight())
                        MB_ImageCells[row_num].set_MB_Color_G(tmpMBG)
                  　# 興奮していなかった場合、ペナルティなし

                    # 青の値を設定
                    if(Judge_excitement[row_num][2]):
                        # 興奮していた場合、代表点のマッチ箱に渡すマッチ棒の数が多かったと考える
                        tmpMBB = MatchBox(MB_ImageCells[row_num].get_MB_Color_B().get_value(), MB_ImageCells[row_num].get_MB_Color_B().get_Weight()-1)
                        MB_ImageCells[row_num].set_MB_Color_B(tmpMBB)

                        # また、本当は興奮してはいけなかったと考える
                        # 値（閾値）を増やす
                        tmpMBB = MatchBox(MB_ImageCells[row_num].get_MB_Color_B().get_value()+1, MB_ImageCells[row_num].get_MB_Color_B().get_Weight())
                        MB_ImageCells[row_num].set_MB_Color_B(tmpMBB)
                  　# 興奮していなかった場合、ペナルティなし

                    # 明るさの値を設定
                    if(Judge_excitement[row_num][3]):
                        # 興奮していた場合、代表点のマッチ箱に渡すマッチ棒の数が多かったと考える
                        tmpMBL = MatchBox(MB_ImageCells[row_num].get_MB_Brightness().get_value(), MB_ImageCells[row_num].get_MB_Brightness().get_Weight()-1)
                        MB_ImageCells[row_num].set_MB_Brightness(tmpMBL)

                        # また、本当は興奮してはいけなかったと考える
                        # 値（閾値）を増やす
                        tmpMBL = MatchBox(MB_ImageCells[row_num].get_MB_Brightness().get_value()+1, MB_ImageCells[row_num].get_MB_Brightness().get_Weight())
                        MB_ImageCells[row_num].set_MB_Brightness(tmpMBL)
                  　# 興奮していなかった場合、ペナルティなし

        # 興奮していなかった場合、ペナルティなし


# 学習データの読込・学習関数の実行を行う関数
# 引数:mode、 1or1以外、 1の時は画面上から学習データを取得する。
#                      1以外の時はファイル読込画面から学習データを取得する。
def run_Learning(mode):
                    
    if(mode == 1):
        # まだnop
    else:
        # 規定のフォルダから正解の学習データを読み込む
                    
                    
                                        
#===============================# 
#===============================#                    
# ループの中で関数を使って学習      #
# ここから学習本編のコードを記載する #
#===============================#
#===============================# 
# 代表点毎のマッチ箱
global MB_ImageCells
                    
# 学習ファイル読込
if(os.path.exists(OUTPUT_FILE_PATH)):
    f = open(OUTPUT_FILE_PATH, 'rb')
    reader = csv.reader(f)

    count = 0
    for row in reader:
        if (count == 0):
            count = count+1
            continue

        # 赤のマッチ箱                    
        MBR = MatchBox(row[1],row[2])
        MB_ImageCells[count-1].set_MB_Color_R(MBR)
                    
        # 緑のマッチ箱
        MBG = MatchBox(row[3],row[4])
        MB_ImageCells[count-1].set_MB_Color_G(MBG)
                    
        # 青のマッチ箱
        MBB = MatchBox(row[5],row[6])
        MB_ImageCells[count-1].set_MB_Color_B(MBB)
                    
        # 明るさののマッチ箱
        MBL = MatchBox(row[7],row[8])
        MB_ImageCells[count-1].set_MB_Brightness(MBL)
                    
        # 代表点のマッチ箱   
        MB = MatchBox(row[9],row[10])
        MB_ImageCells[count-1].set_MB(MB)
                    
        count = count+1

    # csvファイルを閉じる
    f.close()

#========================#
# 学習データを読込ながら学習 #
#========================#
# 学習データ読込
# csvから学習データを読み込み、グローバル変数に格納する

# 学習用の関数を呼び出し

                                                                                
# 書き込み用に上記で指定したファイル名でファイルを開く
f = open(OUTPUT_FILE_PATH, 'w')
writer = csv.writer(f, lineterminator='\n')

# 書き込むデータの準備
# ヘッダー行
str_header_row = ['Pos','R_v','R_w','G_v','G_w','B_v','B_w','L_v','L_w','MB_v','MB_w']
# データを書き込む
writer.writerow(str_header_row)

# ループ処理で値の行を書き込む
for num in range(len(MB_ImageCells)):
    # 書き込むデータの準備
    MBR = MB_ImageCells[num].get_MB_Color_R()
    MBG = MB_ImageCells[num].get_MB_Color_G()
    MBB = MB_ImageCells[num].get_MB_Color_B()
    MBL = MB_ImageCells[num].get_MB_Color_Brightness()
    MB = MB_ImageCells[num].get_MB()
    data = [num,MBR.get_value(),MBR.get_Weight(),MBG.get_value(),MBG.get_Weight(),MBB.get_value(),MBB.get_Weight(),MBL.get_value(),MBL.get_Weight(),MB.get_value(),MB.get_Weight()]

    # データを書き込む
    writer.writerow(data)

# csvファイルを閉じる
f.close()